In [ ]:
# リスト 4.2.1 NLU呼び出し用インスタンス生成（説明コメント付き・.envから資格情報読込版）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の Python SDK を用いた
#     クライアント(NaturalLanguageUnderstandingV1)の初期化を行う。
#   ・APIキー/URL はハードコードせず、.env（またはOS環境変数）から安全に取得する。
#
# セキュリティ/運用の要点:
#   ・ソースコード/リポジトリに認証情報をベタ書きしない（.env を .gitignore に追加する）。
#   ・本番/開発で異なる資格情報を使い分ける場合、.env を切り替えるだけで済む構成にする。
#   ・API バージョンは後方互換に注意。必要に応じて環境変数で上書き可能にしておく。
#
# 事前準備:
#   1) python-dotenv を使用する場合はインストール:
#        pip install python-dotenv
#   2) プロジェクト直下に .env を作成（例）
#        WATSON_NLU_APIKEY=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#        WATSON_NLU_URL=https://api.jp-tok.natural-language-understanding.watson.cloud.ibm.com/instances/xxxxxxxx
#        # （任意）バージョンを固定したい場合
#        # WATSON_NLU_VERSION=2021-08-01
#
#   3) .gitignore に以下を追加して .env をコミットしない:
#        .env
# =============================================================================

import os

# --- .env 読み込み（存在すれば） ------------------------------------------------
try:
    from dotenv import load_dotenv

    # override=False: 既にOS環境に同名キーがある場合は .env を優先しない（誤上書きを防ぐ）
    load_dotenv(override=False)
except ModuleNotFoundError:
    # python-dotenv 未導入でも、OS環境変数から直接取得できるため致命的エラーにはしない
    pass

# --- 環境変数から資格情報を取得 -------------------------------------------------
#   ・キー名の冗長化: 利用環境により命名が違っても拾えるようフォールバックを用意
NLU_APIKEY = (
    os.getenv("WATSON_NLU_APIKEY")
    or os.getenv("NLU_APIKEY")
    or os.getenv("IBM_WATSON_NLU_APIKEY")
)

NLU_URL = (
    os.getenv("WATSON_NLU_URL")
    or os.getenv("NLU_URL")
    or os.getenv("IBM_WATSON_NLU_URL")
)

# 任意: バージョンは環境変数で上書き可能。指定が無ければ従来互換のデフォルトにする
NLU_VERSION = os.getenv("WATSON_NLU_VERSION", "2019-07-12")
# ※ IBM の推奨最新版に合わせたい場合は適宜更新（例: "2021-08-01"）。SDK/サービス側の互換に注意。

# --- バリデーション（早期失敗で原因を明確化） ---------------------------------
missing = []
if not NLU_APIKEY:
    missing.append("WATSON_NLU_APIKEY")
if not NLU_URL:
    missing.append("WATSON_NLU_URL")

if missing:
    raise RuntimeError(
        "NLU の資格情報が不足しています。以下の環境変数を .env または OS 環境に設定してください: "
        + ", ".join(missing)
    )

# --- IBM Watson NLU SDK の初期化 ------------------------------------------------
# SDKのimportは資格情報の検証後に行ってもよい（起動時エラーの責務分離）
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# 旧スタイルのサブモジュール（v1のエクスポート）は必要に応じて:
# from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions, EntitiesOptions

# 認証オブジェクト（IAM）
authenticator = IAMAuthenticator(NLU_APIKEY)

# クライアント生成:
#  - version: API 呼び出しのバージョン日付。サービス仕様と整合する値を用いること。
#  - authenticator: IAMAuthenticator を渡す。
nlu = NaturalLanguageUnderstandingV1(version=NLU_VERSION, authenticator=authenticator)

# サービスURLの設定（インスタンスごとに異なる）
nlu.set_service_url(NLU_URL)

# --- 動作確認の雛形（任意・コメントアウト） -----------------------------------
# from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions
# try:
#     resp = nlu.analyze(
#         text="今日はいい天気ですね。Watsonでキーワード抽出を試します。",
#         features=Features(keywords=KeywordsOptions(limit=3))
#     ).get_result()
#     # print(resp)  # 返却JSON（実運用ではログに個人情報を出さないこと）
# except Exception as e:
#     # ネットワーク/認証/権限/リージョン誤りなどの例外をここで検知
#     raise

In [ ]:
# リスト 4.2.2 NLU呼び出し用共通関数（詳細コメント付き）
# =============================================================================
# 役割:
#   - 事前に初期化済みの IBM Watson NLU クライアント（変数: nlu）を用いて
#     テキスト解析 API を呼び出し、返却 JSON から関心のキーのみを取り出す。
#
# 使い方の要点（理論・API設計の観点）:
#   - IBM Watson NLU では「どの分析を実行するか」を Features オブジェクトで宣言する
#     （例: EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions などを束ねる）。
#   - `nlu.analyze(...)` は HTTP 経由で非同期実行され、`.get_result()` で Python dict(JSON) を返す。
#   - 本関数はその dict から `key` で指定されたトップレベルの要素（例: "entities", "keywords"）を返す。
#     * 存在しないキーを指定すると KeyError となる（後述の安全版で緩和可能）。
#   - ネットワーク障害や認証失敗、レート制限などで例外が発生し得るため、呼び出し側で try/except を推奨。
#
# 例:
#   from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions
#   feats = Features(entities=EntitiesOptions(limit=5, sentiment=False, emotion=False))
#   entities = call_nlu("今日はいい天気ですね。", feats, key="entities")
#   # entities は List[dict]（各エンティティのスパン、タイプ、スコア等を含む）
#
# 注意（実務）:
#   - 入力テキストの長さ・言語・レート制限などサービス側の上限に留意（大きな文書は要分割）。
#   - API のバージョンやリージョンにより挙動が異なる場合があるため、.env で設定を一元管理する。
# =============================================================================

from typing import Any, Dict, List, Union

# Features 型は実行時には SDK から供給されるため、型ヒントとしての参照に留める
# （循環 import を避けるためのフォワード参照も可能）
try:
    from ibm_watson.natural_language_understanding_v1 import (
        Features,
    )  # 型ヒント用（任意）
except Exception:  # ランタイムに存在しない場合でも関数本体の実行には影響しない
    Features = Any  # フォールバック（型チェックツール向けの便宜）


def call_nlu(
    text: str, features: "Features", key: str
) -> Union[Dict[str, Any], List[Any], Any]:
    """
    IBM Watson NLU を呼び出し、返却 JSON から指定キーの要素を取り出して返す薄いヘルパー。

    引数:
        text (str): 解析対象テキスト（UTF-8想定）
        features (Features): 実行する分析機能の宣言（EntitiesOptions/KeywordsOptions 等を束ねたもの）
        key (str): 返却 JSON のトップレベルキー（例: 'entities', 'keywords', 'sentiment', 'categories' など）

    戻り値:
        任意（Union[dict, list, Any]):
            返却 JSON の `key` に対応する値（存在しないキーを指定すると KeyError）

    例外:
        - ネットワーク障害や認証エラー時に SDK 由来の例外が送出される。
        - `key` が返却 JSON に無い場合は KeyError。

    設計メモ:
        - 本関数は「最小限の責務」に留めている（バリデーション/リトライ/ログは呼び出し側で制御）。
        - 実運用では、レート制限や一時的なネットワーク失敗に備えたリトライ・指数バックオフ、
          ならびに返却スキーマ検証（pydantic/dataclasses 等）を併用することを推奨。
    """
    response = nlu.analyze(text=text, features=features).get_result()
    return response[key]


# -----------------------------------------------------------------------------
# （任意）安全版: KeyError の回避・既定キー・軽いバリデーション・簡易リトライを追加する例
# -----------------------------------------------------------------------------
# from ibm_cloud_sdk_core.api_exception import ApiException
# import time
#
# def call_nlu_safe(
#     text: str,
#     features: "Features",
#     key: str | None = None,
#     retries: int = 2,
#     backoff_sec: float = 0.8,
# ) -> Any:
#     """
#     - KeyError を避け、key 未指定時は生の JSON を返す。
#     - 一時的エラーに対して簡易リトライを行う。
#     """
#     last_err = None
#     for attempt in range(retries + 1):
#         try:
#             resp = nlu.analyze(text=text, features=features).get_result()
#             if key is None:
#                 return resp
#             return resp.get(key, None)  # 無ければ None（呼び出し側で判定しやすい）
#         except Exception as e:  # ApiException を個別に握るとより良い
#             last_err = e
#             if attempt < retries:
#                 time.sleep(backoff_sec * (2 ** attempt))  # 指数バックオフ
#                 continue
#             raise last_err
# -----------------------------------------------------------------------------